In [1]:
import pandas as pd
pd.set_option("display.max_rows", None)

In [130]:
from absbox import API, mkDeal

#localAPI = API("http://localhost:8081/", lang="english",check=False)


name = "Name"
dates = {
    "cutoff":"2024-08-31","closing":"2024-08-31","firstPay":"2024-09-30",
    "stated": "2027-09-30",
    "poolFreq": "MonthEnd",
    "payFreq": "MonthEnd"
}

amortising_loan = ["Mortgage"
                    ,{"originBalance": 10925
                    ,"originRate": ["fix",0.22]
                    ,"originTerm": 11
                    ,"freq": "Monthly"
                    ,"type": "even"
                    ,"originDate": "2024-08-31"}
                    ,{"currentBalance": 10925
                    ,"currentRate": 0.22
                    ,"remainTerm": 11
                    ,"status": "current"}
                  ]

bullet_loans = ["Loan"
            ,{"originBalance": 5750
            ,"originRate": ["fix",0.22]
            ,"originTerm": 3
            ,"freq": "Monthly"
            ,"type": "i_p"
            ,"originDate": "2024-08-31"}
            ,{"currentBalance": 5750
            ,"currentRate": 0.22
            ,"remainTerm": 3
            ,"status": "current"}]

j = {"pool_amortizing": {'assets':[amortising_loan]*100},"pool_bullet": {'assets':[bullet_loans]*10}}

accounts = {
    "collections_interest": {"balance": 0},
    "collections_principal_amortizing": {"balance": 0},
    "collections_principal_bullet": {"balance": 0},
    "revolBuyAcc1": {"balance": 0},
    "revolBuyAcc2": {"balance": 0},
    "dealAcc": {"balance": 0},
}

bonds = {
    "A1": {
        "balance": 920_000.00,
        "rate": 0.13,
        "originBalance": 10_000_000,
        "originRate": 0.13,
        "startDate": "2024-08-31",
        "rateType": {"fix": 0.13, "dayCount": "DC_ACT_365A"},
        "maturityDate": "2027-09-30",
        "bondType": {"Sequential":None},
    },
    "EQ": {
        "balance": 230_000,
        "rate": 0.0,
        "originBalance": 230_000,
        "originRate": 0.0,
        "startDate": "2024-08-31",
        "maturityDate": "2027-09-30",
        "rateType": {"Fixed": 0.00},
        "bondType": {"Equity": None},
    },
}

                                

fees = {"utilisation_fee": {"type": {"annualPctFee": [("excess",("originalBondBalance", "A1"),("bondBalance", "A1"),),0.0075],},
                            "feeStart": "2024-12-31"},}
waterfall = {
                    ("DistributionDay", 'Warehousing'):[
                                ["calcInt", "A1"],
                                ["fundWith", "revolBuyAcc1", "A1",{"formula": ("const",475_000)}],
                                ["fundWith", "revolBuyAcc2", "A1",{"formula": ("const",25_000)}],
                                ["fundWith", "revolBuyAcc1", "EQ",{"formula": ("const",118_750)}],
                                ["fundWith", "revolBuyAcc2", "EQ",{"formula": ("const",6250)}],
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ['transfer','collections_principal_amortizing','revolBuyAcc1'],
                                ['transfer','collections_principal_bullet','revolBuyAcc2'],
                                ["transfer","collections_interest","revolBuyAcc1",{
                                "formula": ("-",("bondBalance","A1"),
                                            ("factor",
                                             ("sum",("poolBalance",),
                                              ("accountBalance","revolBuyAcc1","revolBuyAcc2","collections_principal_amortizing","collections_principal_bullet"),),
                                             0.8,),),
                                    "support": ["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]},
                                    ],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc1",None,"Pool1","pool_amortizing"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc2",None,"Pool2","pool_bullet"],
                                ["payIntResidual", "collections_interest", "EQ"],
                                ],
                    "revolving":[
                                ["calcInt", "A1"],
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ['transfer','collections_principal_amortizing','revolBuyAcc1'],
                                ['transfer','collections_principal_bullet','revolBuyAcc2'],
                                ["transfer","collections_interest","revolBuyAcc1",{
                                "formula": ("-",("bondBalance","A1"),
                                            ("factor",
                                             ("sum",("poolBalance",),
                                              ("accountBalance","revolBuyAcc1","revolBuyAcc2","collections_principal_amortizing","collections_principal_bullet"),),
                                             0.8,),)
                                    ,"support": ["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]},
                                    ],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc1",None,"Pool1","pool_amortizing"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc2",None,"Pool2","pool_bullet"],
                                
                                    ["payIntResidual", "collections_interest", "EQ"],
                                ],
                    "amortizing":[
                                ["calcInt", "A1"],  # actions if deal is in Amortizing status
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ['transfer','collections_principal_amortizing','dealAcc'],
                                ['transfer','collections_principal_bullet','dealAcc'],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                                ["transfer","collections_interest","revolBuyAcc1",{
                                "formula": ("-",("bondBalance","A1"),
                                            ("factor",
                                             ("sum",("poolBalance",),
                                              ("accountBalance","revolBuyAcc1","revolBuyAcc2","collections_principal_amortizing","collections_principal_bullet"),),
                                             0.8,),),
                                    "support": ["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]},
                                    ],
                                ["payIntResidual", "collections_interest", "EQ"],
                                ],

                    "cleanUp": [
                            ["sellAsset", ["Current|Defaulted", 1.0, 0], "dealAcc"],
                            ['transfer','collections_principal_amortizing','dealAcc'],
                            ['transfer','collections_principal_bullet','dealAcc'],
                            ["accrueAndPayInt", "collections_interest", ["A1"]],
                            ['transfer','collections_interest','dealAcc'],
                            ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                            ["payPrin", "dealAcc", ["EQ"]],
                            ["payIntResidual", "dealAcc", "EQ"],
                            ],

                    "Defaulted": [
                            ['transfer','collections_principal_amortizing','dealAcc'],
                            ['transfer','collections_principal_bullet','dealAcc'],
                            ["accrueAndPayInt", "collections_interest", ["A1"]],
                            ['transfer','collections_interest','dealAcc'],
                            ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                            ["payIntResidual", "dealAcc", "EQ"],
                            ["payPrin", "dealAcc", ["EQ"]],],
                            
                }

collects = [
            [["pool_amortizing"],"CollectedPrincipal", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedRecoveries", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedPrepayment", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedInterest", "collections_interest"],
            [["pool_bullet"],"CollectedPrincipal", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedRecoveries", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedPrepayment", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedInterest", "collections_interest"],
        ]
defrate3periods = \
   [ ("ratio",("curPoolCollectionStats", None, _*-1, "Defaults"),("curPoolCollectionStats", None, _*-1, "BegBalance"))
   for _ in range(3) ]

trigger = {
            "AfterCollect": {
                "revolving_trigger": {
                    "condition": ["any", [">=", "2026-02-28"]],
                    "effects": ("newStatus", "Revolving"),
                    "status": False,
                    "curable": False,
                },
                "amortization_trigger": {
                    "condition": ["any", [">=", "2026-08-01"]],
                    "effects": ("newStatus", "Amortizing"),
                    "status": False,
                    "curable": False,
                },
                "DefaultRate_trigger": { #Rolling Default rate trigger
            "condition": ["all", 
                          [">=", "2024-12-30"],
                          [("avgRatio",*defrate3periods),">", 0.03167],
            ],
            
            "effects": ("newStatus", "Defaulted"),
            "status": False,
            "curable": False,
        },},
        "BeforeDistribution": {
                "LTV_trigger": {
                    "condition": ["all",
                                  [">", "2024-09-30"],
                                  [
                        (
                            "/",
                            ("bondBalance","A1"),
                            ("sum",
                             ("poolBalance",),
                             ("accountBalance","revolBuyAcc1","revolBuyAcc2","dealAcc","collections_principal_amortizing","collections_principal_bullet"),
                             ),
                        ),
                            ">",
                        0.8,
                    ],],
                    "effects": ("newStatus", "Defaulted"),
                    "status": False,
                    "curable": True,
                },
            }, 
        }

deal_data = {
            "name": name,
            "dates": dates,
            "pool": j,
            "accounts": accounts,
            "fees": fees,
            "bonds": bonds,
            "waterfall": waterfall,
            "collect": collects,
            "trigger": trigger,
            "status": ("PreClosing","Warehousing"),
        }
    
deal = mkDeal(deal_data)
revol_asset1 = amortising_loan
revol_asset2 = bullet_loans

cdr_assump = 0.0

cdr_assump_1 = 0.0

revolvingPool1 = (["constant",revol_asset1]
                 ,("Pool",("Mortgage",{"CDR":cdr_assump_1},None,None,None
                                                                   )
                   ,None
                   ,None))

revolvingPool2 = (["constant",revol_asset2]
                 ,("Pool",("Loan",{"CDR":cdr_assump_1},None,None,None)
                   ,None
                   ,None))

amortisingPoolAssump = ("Pool",("Mortgage",{"CDR":cdr_assump},None,None,None)
                   ,None
                   ,None)

bulletPoolAssump = ("Pool",("Loan",{"CDR":cdr_assump},None,None,None)
                   ,None
                   ,None)

from absbox import EnginePath

localAPI = API(EnginePath.LOCAL, lang="english", check=False)
r = localAPI.run(deal,
                 poolAssump = ("ByPoolId",
                                    {"pool_amortizing":amortisingPoolAssump
                                    ,"pool_bullet":bulletPoolAssump}
                                    ),
         runAssump=[#tuple(trigger_inspect_creation(deal)),
             ("inspect",("MonthEnd",("bondBalance", "A1")),
              ("MonthEnd",("poolBalance","pool_amortizing")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Principal")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Interest")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Defaults"))
              ,("MonthEnd",("accountBalance","revolBuyAcc1","revolBuyAcc2",))
                              ,("MonthEnd",("curPoolCollectionStats", None, 0, "BegBalance"))
                              ,("MonthEnd",("ratio",("curPoolCollectionStats", None, 0, "Defaults"),("curPoolCollectionStats", None, 0, "BegBalance")))
                              ,("MonthEnd",("ratio",("curPoolCollectionStats", None, -1, "Defaults"),("curPoolCollectionStats", None, -1, "BegBalance")))
                              ,("MonthEnd",("ratio",("curPoolCollectionStats", None, -2, "Defaults"),("curPoolCollectionStats", None, -2, "BegBalance")))
                              
                              ),
                              ("call", {"afterDate":"2027-08-01"}),
                    ("revolving"
                       ,{"Pool1":revolvingPool1,"Pool2":revolvingPool2})
             ,("stop","2024-11-01")
         ]
         ,read=True)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.29.9

Warning Message from server:
Bond A1 is not paid off
Bond EQ is not paid off

In [131]:
r['pool']['flow']['pool_amortizing']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-08-31,1092500.00,0.00,0.00,0,0,0,0,0.22,None,None,0.00,0,0,0,0,0
2024-09-30,993182.00,99318.00,20028.00,0,0,0,0,0.22,None,None,99318.00,0,0,0,0,0
2024-10-31,1523925.82,162324.18,30914.01,0,0,0,0,0.22,None,None,261642.18,0,0,0,0,0


In [132]:
r['result']['status']

,Date,From,To
0,2024-08-31,PreClosing,Warehousing
1,2024-11-01,DealEnd,Stop Run Flag


In [133]:
r['result']['inspect'][ "<CurrentPoolBalance:{'tag': 'PoolName', 'contents': 'pool_amortizing'}>"]

,"<CurrentPoolBalance:{'tag': 'PoolName', 'contents': 'pool_amortizing'}>"
Date,
2024-08-31,1092500.0
2024-09-30,1686250.0
2024-10-31,2280000.0
